In [1]:
# github에서 이 책의 예시 프로젝트를 다운받습니다.
!git clone https://github.com/baek2sm/ml.git

Cloning into 'ml'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 37 (delta 3), reused 31 (delta 2), pack-reused 0
Unpacking objects: 100% (37/37), done.


In [2]:
# 실습에 필요한 라이브러리를 불러옵니다.
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import joblib

In [3]:
# 서울 기상 데이터 세트를 불러옵니다.
datasets = joblib.load('./ml/datasets/weather.pickle')

# 입력 데이터와 타깃을 준비합니다.
data, labels = datasets['data'], datasets['labels']

# 입력 데이터와 타깃의 형태를 출력합니다.
print(data.shape, labels.shape)

(26181, 24, 3) (26181,)


In [4]:
# 학습 세트와 테스트 세트를 나눕니다.
train_length = 20000
X_train, X_test = data[:train_length], data[train_length:]
y_train, y_test = labels[:train_length], labels[train_length:]

# 넘파이 자료구조에서 텐서 자료구조로 변경합니다.
X_train, X_test = torch.from_numpy(X_train).to('cuda'), torch.from_numpy(X_test).to('cuda')
y_train, y_test = torch.from_numpy(y_train).to('cuda'), torch.from_numpy(y_test).to('cuda')

In [12]:
# 데이터 세트 자료구조를 준비합니다.
dset_train, dset_test = TensorDataset(X_train, y_train), TensorDataset(X_test, y_test)

# 데이터 로더를 준비합니다.
train_loader = DataLoader(dset_train, batch_size=256, shuffle=True)
test_loader = DataLoader(dset_test, batch_size=256, shuffle=False)

In [6]:
# RNN 모델 클래스를 정의합니다.
class RNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.cell = nn.RNN(3, 3, batch_first=True)
    self.fc = nn.Linear(24*3, 1)

  def forward(self, X):
    out, hidden_state = self.cell(X)
    out = out.contiguous()
    out = self.fc(out.view(-1, 24*3))
    return out

In [11]:
# RNN 모델 객체를 생성합니다.
model = RNN().to('cuda')

# 평균 제곱 오차(Mean Squared Error) 손실 함수 객체를 생성합니다.
criterion = nn.MSELoss().to('cuda')

# 아담 옵티마이저 객체를 생성합니다.
optimizer = optim.Adam(model.parameters(), lr=2e-4)

In [ ]:
from torchsummary import summary
summary(model, (24, 3))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
               RNN-1  [[-1, 24, 5], [-1, 2, 5]]               0
            Linear-2                    [-1, 1]             121
Total params: 121
Trainable params: 121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.00
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


In [8]:
# 학습 함수를 정의합니다.
def train(model, criterion, optimizer, loader):
  # 현재 에포크의 오차를 저장할 변수를 생성합니다.
  epoch_loss = 0

  # 모델을 학습 모드로 설정합니다.
  model.train()

  # 배치 학습을 실행합니다.
  for X_batch, y_batch in loader:
    # 입력 데이터와 타깃을 준비합니다.
    X_batch, y_batch = X_batch.to('cuda'), y_batch.to('cuda').float().view(-1, 1)
    # 기울기를 초기화합니다.
    optimizer.zero_grad()
    # 모델을 사용해 타깃을 예측합니다.
    hypothesis = model(X_batch)
    # 손실 함수로 오차를 계산합니다.
    loss = criterion(hypothesis, y_batch)        
    # 기울기를 계산합니다.
    loss.backward()
    # 경사 하강법으로 가중치를 수정합니다.
    optimizer.step()    
    # 현재 배치의 오차를 저장합니다.
    epoch_loss += loss.item()

  # 현재 에포크의 오차를 반환합니다.
  return epoch_loss / len(loader)

In [9]:
# 평가 함수를 정의합니다.
def evaluate(model, criterion, optimizer, loader):
  # 현재 에포크의 오차를 저장할 변수를 생성합니다.
  epoch_loss = 0

  # 모델을 평가 모드로 설정합니다.
  model.eval()

  with torch.no_grad():
    # 배치 학습을 실행합니다.
    for X_batch, y_batch in loader:
      # 입력 데이터와 타깃을 그래픽카드로 연산하도록 준비합니다.
      X_batch, y_batch = X_batch.to('cuda'), y_batch.to('cuda').float().view(-1, 1)
      # 모델을 사용해 타깃을 예측합니다.
      hypothesis = model(X_batch)
      # 손실 함수로 오차를 계산합니다.
      loss = criterion(hypothesis, y_batch)
      # 현재 배치의 오차를 저장합니다.
      epoch_loss += loss.item()

    # 현재 에포크의 오차를 반환합니다.
    return epoch_loss / len(loader)

In [13]:
# 200회에 걸쳐 모델을 학습시킵니다.
for epoch in range(1, 201):
  # 모델을 학습시킵니다.
  loss = train(model, criterion, optimizer, train_loader)

  # 모델을 평가합니다.
  test_loss = evaluate(model, criterion, optimizer, test_loader)

  if epoch % 20 == 0:
    # 20 에포크마다 현재 에포크의 학습 결과를 출력합니다.
    print('epoch: {}, loss: {:.3f}, test_loss: {:.3f}'.format(
        epoch, loss, test_loss
    )) 

epoch: 20, loss: 48.659, test_loss: 31.791
epoch: 40, loss: 24.080, test_loss: 26.174
epoch: 60, loss: 17.486, test_loss: 18.247
epoch: 80, loss: 14.357, test_loss: 15.416
epoch: 100, loss: 12.480, test_loss: 13.896
epoch: 120, loss: 11.621, test_loss: 13.233
epoch: 140, loss: 11.058, test_loss: 12.684
epoch: 160, loss: 10.528, test_loss: 12.300
epoch: 180, loss: 10.233, test_loss: 12.008
epoch: 200, loss: 9.881, test_loss: 11.785


In [14]:
# 모델을 평가 모드로 설정합니다.
model.eval()
with torch.no_grad():    
    # 테스트 세트의 데이터로더에서 배치 단위로 데이터를 불러옵니다.
    for X_batch, y_batch in test_loader:
      # 배치 데이터를 cuda로 이동시키고, 타깃의 타입과 형태를 조정합니다.
      X_batch, y_batch = X_batch.to('cuda'), y_batch.to('cuda').float().view(-1, 1)
      # 모델을 사용해 타깃을 예측합니다. 예측값은 10개 클래스에 대한 확률값입니다.
      hypothesis = model(X_batch)      
      # 배치의 첫 번째 데이터 샘플마다 모델이 예측한 값과 실제 타깃을 출력합니다.
      print('predicted target: {:.2f}, real target: {:.2f}'.format(hypothesis[0].item(), y_batch[0].item()))

predicted target: 17.82, real target: 18.40
predicted target: 9.46, real target: 5.20
predicted target: 12.86, real target: 11.50
predicted target: 5.52, real target: 8.90
predicted target: 3.06, real target: -0.20
predicted target: -2.09, real target: 0.20
predicted target: 2.37, real target: 0.30
predicted target: 0.08, real target: 5.00
predicted target: 3.13, real target: -1.20
predicted target: 2.46, real target: 3.60
predicted target: 3.65, real target: 2.70
predicted target: 0.70, real target: 5.20
predicted target: 4.32, real target: 8.20
predicted target: 0.98, real target: 0.50
predicted target: 8.56, real target: 3.60
predicted target: 14.72, real target: 16.00
predicted target: 4.57, real target: 7.30
predicted target: 16.29, real target: 13.20
predicted target: 12.53, real target: 15.80
predicted target: 15.33, real target: 11.70
predicted target: 14.91, real target: 17.70
predicted target: 19.35, real target: 21.60
predicted target: 19.38, real target: 19.30
predicted tar